In [156]:
import os,re
import pandas as pd
import jieba
from tqdm import tqdm
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

# load data
## 上個表df

In [2]:
ROOT = "../02preprocess/news_found_keyterms/"
thisFile = os.listdir(ROOT)[-1]
target = ROOT+"{}".format(thisFile)
print(target)
df = pd.read_pickle(target)
content_s = df.content

['news_found_keyterms_20190818']


'../02preprocess/news_found_keyterms/news_found_keyterms_20190818'

## 停用詞

In [84]:
sentDictFolder = "../00ref/stopwords//"
allFile = os.listdir(sentDictFolder)
stopWord_set = set()

for afile in allFile:
    target = "{}{}".format(sentDictFolder,afile)
    with open(target,encoding = "utf8") as f:
        notList = f.readlines()
        strlist = map(str.strip,notList[1:])
        stopWord_set.update(strlist)
        
stopWord_set = list(stopWord_set)

# 塞進 vectorizer 之前的前處理

In [200]:
tokenizedContent_s=content_s.apply(str.strip).progress_apply(jieba.cut)
removeDot_s = tokenizedContent_s.progress_apply(lambda aList:[n.replace(".","") for n in aList])
cleanedContent_s = removeDot_s.progress_apply(lambda aList : [n.strip() for n in aList if ((not bool(re.search(r'\d', n))) and (not n in stopWord_set))])
joinedTokens_s = cleanedContent_s.progress_apply(" ".join)
joinedTokens_s.head()

11279    年 月份 股指 大涨 统计数据 显示 越来越 选择 清仓 离场 月份 A股 持仓 账户 减少...
11274    国务院 生产 流通 环节 水电 价格 运营 费用 规范 降低 农产品 市场 收费 强化 零售...
11272    月 日 茅台 集团 官网 发布 声明 国家 发改委 价格 监督 检查 反垄断 局 贵州省 物...
11271    启动 NFC SWP 试点 规模 投放 NFC SWP 卡 启动 终端 定制 中国电信  董...
11270    昨日 沪 深 两市 双双 高开 沪 指 放量 创出 近期 新高 收报 点 成交 亿元 放量 ...
Name: content, dtype: object

In [232]:
df['joinedTokens_s'] = joinedTokens_s

# 塞進 vectorizer

In [296]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(joinedTokens_s)
idf = vectorizer.idf_
term = vectorizer.get_feature_names()
idfDict = dict(zip(term, idf))

In [234]:
idfDict

{'aa': 6.1937501869096065,
 'aaa': 6.827473946998751,
 'abb': 6.994528031661917,
 'abs': 7.252357140964016,
 'ac': 7.8883459076840134,
 'adas': 7.141131505853792,
 'adc': 8.293811015792178,
 'adr': 8.006128943340396,
 'advanced': 8.13966033596492,
 'ag': 7.252357140964016,
 'agv': 7.8883459076840134,
 'ah': 7.141131505853792,
 'ai': 5.911183215124596,
 'all': 7.687675212221863,
 'alpha': 7.8883459076840134,
 'alphago': 8.293811015792178,
 'amazon': 7.7829853920261876,
 'amc': 7.8883459076840134,
 'amd': 8.293811015792178,
 'amoled': 7.04104804729681,
 'and': 7.8883459076840134,
 'anda': 8.293811015792178,
 'android': 7.141131505853792,
 'api': 8.293811015792178,
 'app': 5.666729877223635,
 'apple': 6.950076269091083,
 'applewatch': 8.006128943340396,
 'ar': 6.5302224235308195,
 'arm': 7.8883459076840134,
 'arpu': 7.446513155404975,
 'as': 8.13966033596492,
 'asa': 8.293811015792178,
 'asset': 8.13966033596492,
 'at': 8.006128943340396,
 'atm': 7.377520283918023,
 'avc': 8.2938110157921

# 計算他的情緒因子

In [220]:
from sklearn import preprocessing
ntusdNegInContent = df.ntusdNegInContent
NegCountMean = ntusdNegInContent.apply(len).mean()
NegCountStd = ntusdNegInContent.apply(len).std()
NegCountNorm = (ntusdNegInContent.apply(len)-NegCountMean)/NegCountStd

ntusdPosInContent = df.ntusdPosInContent
PosCountMean = ntusdPosInContent.apply(len).mean()
PosCountStd = ntusdPosInContent.apply(len).std()
PosCountNorm = (ntusdPosInContent.apply(len)-PosCountMean)/PosCountStd

df['sentimentScore'] = PosCountNorm-NegCountNorm
df['sentimentScore'].head()

11279   -1.301306
11274    0.553302
11272   -0.873620
11271    0.756551
11270   -0.623811
Name: sentimentScore, dtype: float64

In [239]:
superGoodTokens = df.sort_values(by = 'sentimentScore',ascending=False)[:20].joinedTokens_s
superGoodVectors = vectorizer.transform(superGoodTokens)
superGoodVectors.shape

(20, 26267)

In [299]:
import numpy as np
tfidfSum_array = np.asarray(np.sum(superGoodVectors,axis = 0))[0]
superGoodTerms = np.asarray(term)[np.argsort(tfidfSum_array)][-100:]

In [300]:
for term in superGoodTerms:
    print(term,idfDict[term])

门店 5.4316101348627095
机制 3.484340366291357
有望 2.167305798924959
物流 3.9097032149144715
布局 2.6136384067751104
指出 2.2833611687731152
交易 2.3384055184998984
国际 2.520295203121302
会议 3.241820536014008
机器人 3.9897459225880083
深市 5.068290648205308
加快 2.8608170792522616
改善 3.11954629791412
收入 2.313660228584469
生态 3.8033037381242263
电站 5.470449968178974
责任 4.213452695544817
沪市 4.876084332178812
预计 2.007192381393714
证监会 3.081596348297553
vr 5.642919228529917
股权 2.413743687141452
产品 2.129444133806733
深铁 7.446513155404975
升级 3.0441588212254223
信息 2.325529804139853
推动 2.5470754856067677
医院 4.733712351544167
未来 1.890788846576874
质量 3.5576125673976824
合作 2.697242625443126
供给 3.52453981912366
临时 4.498321826619984
科创板 5.596934115288093
同比 2.184933872002228
战略 2.557776497298789
港通 4.757694316230651
建设 2.3989491500756133
股东 2.0281929964622414
内地 4.6606204675388305
模式 3.009761621041846
净利润 2.1489103297948065
康复 6.347900866736865
债券市场 5.768082371483922
提高 2.738270784277069
香港 3.777472043510702
投资 1.6200917292